<a href="https://colab.research.google.com/github/Codestronomer/HamoyeAI_Internship/blob/main/notebooka58486fc95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import keras.models as models
from keras import backend

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')

train_df.head()

FileNotFoundError: ignored

In [ ]:
train_df['tags'].values

In [ ]:
train_df.describe()

In [ ]:
test_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')

test_df.head()

In [ ]:
test_df['tags'].values

In [ ]:
test_df.describe()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
# fig, ax = plt.subplots()

# ax.bar(train_df['tags'], train_df['image_name'], width=1, edgecolor="white", linewidth=0.7)

# plt.show()

In [ ]:
def tag_mapping(tag_csv):
    '''
    returns dictionary that maps tags to integers, and another that 
    maps integers to tags (inverse).
    '''
    
    all_tags = set()
    
    for i in range(len(tag_csv)):
        
        # convert spaced separated tags into an array of tags
        tags = tag_csv['tags'][i].split(' ')
        all_tags.update(tags)
    all_tags = list(all_tags)
    
    # sort alphabetically
    all_tags.sort()
    
    # dict that maps tags to integers, and the reverse
    tags_map = {all_tags[i]:i for i in range(len(all_tags))}
    inv_tags_map = {i:all_tags[i] for i in range(len(all_tags))}
    
    return tags_map, inv_tags_map

In [ ]:
def file_mapping(tag_csv):
    '''
    returns dictionary with image names as keys and split tags as values.
    '''
    mapping = dict()
    for i in range(len(tag_csv)):
        
        name, tags = tag_csv['image_name'][i], tag_csv['tags'][i]
        mapping[name] = tags.split(' ')
        
    return mapping

In [ ]:
def one_hot_encode(tags, mapping):
    encoding = np.zeros(len(mapping), dtype='uint8')
    
    for tag in tags:
        encoding[mapping[tag]] = 1
        
    return encoding

In [ ]:
def make_dataset(path,file_mapping,tag_mapping):
    photos,targets=list(),list()
    for filename in os.listdir(path):
        photo=load_img(path+filename,target_size=(75,75))
        photo=img_to_array(photo,dtype='uint8')
        tags=file_mapping[filename[:-4]]
        target=one_hot_encode(tags,tag_mapping)
        photos.append(photo)
        targets.append(target)
    X=np.asarray(photos,dtype='uint8')
    y=np.asarray(targets,dtype='uint8')
    return X,y

In [ ]:
tags_mapping,_=tag_mapping(train_df)
files_mapping=file_mapping(train_df)
path='/kaggle/input/planets-dataset/planet/planet/train-jpg/'
X,y=make_dataset(path,files_mapping,tags_mapping)

print('X shape:', X.shape, 'y shape:', y.shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# make all one predictions
train_yhat = np.asarray([np.ones(y_train.shape[1]) for _ in range(y_train.shape[0])])
test_yhat = np.asarray([np.ones(y_test.shape[1]) for _ in range(y_test.shape[0])])

In [ ]:
train_score = fbeta_score(y_train, train_yhat, beta=2, average='samples')
test_score = fbeta_score(y_test, test_yhat, beta=2, average='samples')
print('All Ones: train=%.3f, test=%.3f' % (train_score, test_score))

An fbeta score function that can be used by keras



In [ ]:
def calculate_fbeta_score(y_true, y_pred, beta=2):
    y_pred = backend.clip(y_pred, 0, 1)

    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    p = tp / (tp + fp + backend.epsilon())
    r = tp / (tp + fn + backend.epsilon())
    bb = beta ** 2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score

In [ ]:
import tensorflow as tf 
from tensorflow.compat.v1 import Session as sess 
tf.compat.v1.disable_eager_execution()

In [ ]:
# test calculate_fbeta_score - to ensure result is equal to sk-learn result
train_score = calculate_fbeta_score(backend.variable(y_train), backend.variable(train_yhat)).eval(session=backend.get_session())
test_score = calculate_fbeta_score(backend.variable(y_test), backend.variable(test_yhat)).eval(session=backend.get_session())
print('All Ones (keras): train=%.3f, test=%.3f' % (train_score, test_score))

In [ ]:
from keras.applications import InceptionV3
model=InceptionV3(input_shape=(75,75,3),include_top=False)
for layer in model.layers:
    layers.trainable=False
last_layer=model.get_layer('mixed7')
last_output=last_layer.output
    
x=layers.Flatten()(last_output)
x=layers.Dense(1024,activation='relu')(x)
x=layers.Dense(512,activation='relu')(x)
x=layers.Dropout(0.2)(x)
x=layers.Dense(17,activation='sigmoid')(x)
model=models.Model(model.inputs,x)
model.compile(optimizer=RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=[fbeta])

In [ ]:
train_datagen=ImageDataGenerator(rescale=1.0/255.0,horizontal_flip=True, vertical_flip=True, rotation_range=90)
test_datagen=ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train_gen=train_datagen.flow(X_train,y_train,batch_size=64)
test_gen=test_datagen.flow(X_test,y_test,batch_size=64)

In [ ]:
history = model.fit(train_gen,steps_per_epoch=500,validation_data=test_gen, validation_steps=127, epochs=250, verbose=0)

In [ ]:
loss, fbeta =model.evaluate_generator(test_gen, steps=8, verbose=0)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
history.history['fbeta']

In [ ]:
test_path_1='/kaggle/input/planets-dataset/planet/planet/test-jpg/'
test_path_2='/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional/'

In [ ]:
submission_df=pd.read_csv('/kaggle/input/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv')

In [ ]:
photo_test=[]
for filename in submission_df['image_name']:
    if filename[:1]=='t':
        img=load_img(test_path_1+filename+'.jpg',target_size=(75,75))
    elif filename[:1]=='f':
        img=load_img(test_path_2+filename+'.jpg',target_size=(75,75))
    ph=img_to_array(img,dtype='uint8')
    photo_test.append(ph)

In [ ]:
test_x=np.asarray(photo_test,dtype='uint8')

image_gen_test=ImageDataGenerator(rescale=1/255.0)
test_data_gen=image_gen_test.flow(test_x,shuffle=False,batch_size=64)

In [ ]:
result=model.predict(test_data_gen)

In [ ]:
new_df=pd.DataFrame(result,columns=tags_mapping.keys())

In [ ]:
tags=new_df.columns
pred_tags=new_df.apply(lambda x: ' '.join(tags[x>0.5]),axis=1)

In [ ]:
pred_tag=pd.DataFrame(pred_tags,columns=['tags'])

In [ ]:
submission_df['tags']=pred_tag['tags']

submission_df.to_csv('attempt_4.csv',index=False)

submission_df.head()